In [1]:
import pandas as pd
from pathlib import Path

# Define the path to the 'Resources' folder
# Adjust the path as necessary based on your project's structure
resources_path = Path("Resources")

# Check if the 'Resources' folder exists
if not resources_path.exists():
    # If the folder does not exist, print an error message
    print(f"Error: The folder '{resources_path}' does not exist.")
else:
    # Use glob to find all CSV files in the 'Resources' folder
    # The pattern '*.csv' matches all files with a .csv extension
    csv_files = list(resources_path.glob("*.csv"))
    
    # Check if any CSV files were found
    if not csv_files:
        # If no CSV files are found, print an error message
        print("Error: No CSV files found in the 'Resources' folder.")
    else:
        # Initialize an empty list to store cleaned DataFrames
        # This will be used to concatenate all cleaned DataFrames later
        cleaned_dataframes = []
        
        # Iterate over each CSV file found
        for csv_file in csv_files:
            try:
                # Read the CSV file into a DataFrame using pandas
                # low_memory=False helps in efficiently reading large files by processing them in chunks
                df = pd.read_csv(csv_file, low_memory=False)
                
                # Print a success message indicating the file was read
                print(f"Successfully read '{csv_file.name}'")
                
                # -------------------- Data Cleaning Steps --------------------
                
                # 1. Replace empty strings or strings with only whitespace with NaN
                # This standardizes missing or undefined data for easier processing
                df.replace(r'^\s*$', pd.NA, regex=True, inplace=True)
                
                # 2. Convert 'started_at' and 'ended_at' columns to datetime format
                # This ensures that date and time operations can be performed on these columns
                # Errors='coerce' will convert invalid parsing to NaT (Not a Time)
                df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
                df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
                
                # 3. Drop rows with any missing values, including NaN and NaT
                # This ensures that the dataset only contains complete records
                df.dropna(inplace=True)
                
                # 4. Sample 500 rows if there are at least 500; otherwise, use all rows
                # This helps in managing memory usage and ensures consistency in dataset size
                if len(df) >= 500:
                    df_sampled = df.sample(n=500, random_state=42)  # random_state for reproducibility
                else:
                    df_sampled = df.copy()  # Use all rows if less than 500
                
                # 5. Append the cleaned and sampled DataFrame to the list
                cleaned_dataframes.append(df_sampled)
                
                # Print a message indicating that cleaning was successful for this file
                print(f"Successfully cleaned and sampled '{csv_file.name}'")
                
            except Exception as e:
                # If an error occurs while reading or processing the file, print an error message
                print(f"Error processing '{csv_file.name}': {e}")
        
        # After processing all files, check if there are any cleaned DataFrames to concatenate
        if cleaned_dataframes:
            try:
                # Concatenate all cleaned DataFrames into a single DataFrame
                # ignore_index=True resets the index in the resulting DataFrame
                tripdata_df = pd.concat(cleaned_dataframes, ignore_index=True)
                
                # Print a success message indicating the DataFrames were concatenated
                print("\nAll cleaned DataFrames have been successfully combined into 'tripdata_df'.")
                
                # Optionally, display the first few rows of the combined DataFrame
                print("\nSample of the combined DataFrame:")
                print(tripdata_df.head())
                
                # Optionally, display the shape of the combined DataFrame
                print(f"\nTotal records in 'tripdata_df': {tripdata_df.shape[0]}")
                
            except Exception as e:
                # If an error occurs during concatenation, print an error message
                print(f"Error concatenating DataFrames: {e}")
        else:
            # If no DataFrames were cleaned and collected, inform the user
            print("No DataFrames were cleaned and collected. 'tripdata_df' was not created.")
# Display the tripdate
tripdata_df.info()

Successfully read '202309-citibike-tripdata_1.csv'
Successfully cleaned and sampled '202309-citibike-tripdata_1.csv'
Successfully read '202310-citibike-tripdata_1.csv'
Successfully cleaned and sampled '202310-citibike-tripdata_1.csv'
Successfully read '202311-citibike-tripdata_1.csv'
Successfully cleaned and sampled '202311-citibike-tripdata_1.csv'
Successfully read '202312-citibike-tripdata_1.csv'
Successfully cleaned and sampled '202312-citibike-tripdata_1.csv'
Successfully read '202401-citibike-tripdata.csv'
Successfully cleaned and sampled '202401-citibike-tripdata.csv'
Successfully read '202402-citibike-tripdata.csv'
Successfully cleaned and sampled '202402-citibike-tripdata.csv'
Successfully read '202403-citibike-tripdata.csv'
Successfully cleaned and sampled '202403-citibike-tripdata.csv'
Successfully read '202404-citibike-tripdata.csv'
Successfully cleaned and sampled '202404-citibike-tripdata.csv'
Successfully read '202405-citibike-tripdata_1.csv'
Successfully cleaned and samp

In [2]:
tripdata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   ride_id                                6000 non-null   object        
 1   rideable_type                          6000 non-null   object        
 2   started_at                             6000 non-null   datetime64[ns]
 3   ended_at                               6000 non-null   datetime64[ns]
 4   start_station_name                     6000 non-null   object        
 5   start_station_id                       6000 non-null   object        
 6   end_station_name                       6000 non-null   object        
 7   end_station_id                         6000 non-null   object        
 8   start_lat                              6000 non-null   float64       
 9   start_lng                              6000 non-null   float64 

In [3]:
# -------------------- Separate 'started_at' and 'ended_at' into Date and Time Components --------------------

# 1. Create a new column 'started_at_date' by extracting only the date part and converting to datetime64[ns]
# We use .dt.floor('D') to get just the date part and ensure it's datetime64[ns]
tripdata_df['started_at_date'] = tripdata_df['started_at'].dt.floor('D')

# 2. Create a new column 'started_at_time' by extracting the time part
# Pandas does not support a datetime64[ns] type for time only, so we leave it as an object or string
tripdata_df['started_at_time'] = tripdata_df['started_at'].dt.strftime('%H:%M:%S')

# 3. Create a new column 'ended_at_date' by extracting only the date part and converting to datetime64[ns]
tripdata_df['ended_at_date'] = tripdata_df['ended_at'].dt.floor('D')

# 4. Create a new column 'ended_at_time' by extracting the time part
tripdata_df['ended_at_time'] = tripdata_df['ended_at'].dt.strftime('%H:%M:%S')

# -------------------- Optional: Verify the New Columns --------------------

# Print a confirmation message to indicate that the separation was successful
print("\nSuccessfully separated 'started_at' and 'ended_at' into date (datetime64[ns]) and time columns.")

# Display the first few rows of the DataFrame to verify the new columns
print("\nSample of 'tripdata_df' with new date and time columns:")
print(tripdata_df[['started_at', 'started_at_date', 'started_at_time', 
                   'ended_at', 'ended_at_date', 'ended_at_time']].head())

# Optionally, display the data types of the new columns to confirm their types
print("\nData types of the new date and time columns:")
print(tripdata_df[['started_at_date', 'started_at_time', 
                   'ended_at_date', 'ended_at_time']].dtypes)



Successfully separated 'started_at' and 'ended_at' into date (datetime64[ns]) and time columns.

Sample of 'tripdata_df' with new date and time columns:
               started_at started_at_date started_at_time  \
0 2023-09-05 13:08:38.032      2023-09-05        13:08:38   
1 2023-09-30 14:42:24.920      2023-09-30        14:42:24   
2 2023-09-05 08:03:53.556      2023-09-05        08:03:53   
3 2023-09-30 14:48:52.633      2023-09-30        14:48:52   
4 2023-09-20 14:27:41.509      2023-09-20        14:27:41   

                 ended_at ended_at_date ended_at_time  
0 2023-09-05 13:23:05.790    2023-09-05      13:23:05  
1 2023-09-30 14:44:12.437    2023-09-30      14:44:12  
2 2023-09-05 08:29:05.885    2023-09-05      08:29:05  
3 2023-09-30 14:58:15.976    2023-09-30      14:58:15  
4 2023-09-20 14:33:14.101    2023-09-20      14:33:14  

Data types of the new date and time columns:
started_at_date    datetime64[ns]
started_at_time            object
ended_at_date      datetime64

In [4]:
tripdata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   ride_id                                6000 non-null   object        
 1   rideable_type                          6000 non-null   object        
 2   started_at                             6000 non-null   datetime64[ns]
 3   ended_at                               6000 non-null   datetime64[ns]
 4   start_station_name                     6000 non-null   object        
 5   start_station_id                       6000 non-null   object        
 6   end_station_name                       6000 non-null   object        
 7   end_station_id                         6000 non-null   object        
 8   start_lat                              6000 non-null   float64       
 9   start_lng                              6000 non-null   float64 

In [9]:
# -------------------- Drop Unwanted Columns --------------------

# 1. Drop the 'Unnamed' and 'rideable_type_duplicate_column_name_1' columns if they exist
# Use errors='ignore' to avoid errors if the columns don't exist in the DataFrame
tripdata_df.drop(columns=['Unnamed: 0', 'rideable_type_duplicate_column_name_1'], errors='ignore', inplace=True)

# Print a confirmation message that the columns have been dropped
print("\nDropped columns: 'Unnamed: 0' and 'rideable_type_duplicate_column_name_1'.")

# -------------------- Save the Cleaned DataFrame to a CSV File --------------------

# 2. Save the cleaned DataFrame to a CSV file named 'citibike_tripdata.csv'
# index=False prevents pandas from writing the row index to the CSV file
tripdata_df.to_csv('citibike_tripdata.csv', index=False)

# Print a confirmation message indicating that the DataFrame has been saved
print("\nDataFrame has been successfully saved to 'citibike_tripdata.csv'.")

tripdata_df.info()



Dropped columns: 'Unnamed: 0' and 'rideable_type_duplicate_column_name_1'.

DataFrame has been successfully saved to 'citibike_tripdata.csv'.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ride_id             6000 non-null   object        
 1   rideable_type       6000 non-null   object        
 2   started_at          6000 non-null   datetime64[ns]
 3   ended_at            6000 non-null   datetime64[ns]
 4   start_station_name  6000 non-null   object        
 5   start_station_id    6000 non-null   object        
 6   end_station_name    6000 non-null   object        
 7   end_station_id      6000 non-null   object        
 8   start_lat           6000 non-null   float64       
 9   start_lng           6000 non-null   float64       
 10  end_lat             6000 non-null   float64       
 11  end_lng          